In [102]:
import pandas as pd
from redfin import Redfin
import requests
import os
from pprint import PrettyPrinter

pp = PrettyPrinter(indent=1)

# Replace 'your_file.csv' with the path to your CSV file
file_path = 'Addresses_cleaned.csv'

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df.head()


,ACCOUNT_NUM,DIVISION_CD,STREET_NUM,FULL_STREET_NAME,PROPERTY_CITY,PROPERTY_ZIPCODE,GIS_PARCEL_ID
0,38204500060410000,RES,2326,LUAU ST,MESQUITE (DALLAS CO),751503826,38204500060410000
1,38137500370290000,RES,1737,HIGHLAND ST,MESQUITE (DALLAS CO),751491534,38137500370290000
2,60259500060160000,RES,9999,AUTUMN WOOD DR,MESQUITE (DALLAS CO),75181,60259500060160000
3,60259500050250000,RES,9925,AUTUMN WOOD DR,MESQUITE (DALLAS CO),75181,60259500050250000
4,60259500060230000,RES,9999,RIPPLEWOOD DR,MESQUITE (DALLAS CO),75181,60259500060230000


In [103]:
# Remove unnesessary (DALLAS CO) from PROPERTY_CITY series
df['PROPERTY_CITY'] = df['PROPERTY_CITY'].str.replace(r'\(DALLAS CO\)', '').str.rstrip()

# Add a column with the full address
df['FullAddress'] = df['STREET_NUM'].astype(str) + ' ' + df['FULL_STREET_NAME'] + ' ' + df['PROPERTY_CITY'] + ' ' + 'TX' + ' ' + df['PROPERTY_ZIPCODE'].astype(str).str[:5]
df.head()


/var/folders/lm/zm6_v2d953zcr8t5mrvy0_qw0000gn/T/ipykernel_77851/4063174521.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['PROPERTY_CITY'] = df['PROPERTY_CITY'].str.replace(r'\(DALLAS CO\)', '').str.rstrip()


,ACCOUNT_NUM,DIVISION_CD,STREET_NUM,FULL_STREET_NAME,PROPERTY_CITY,PROPERTY_ZIPCODE,GIS_PARCEL_ID,FullAddress
0,38204500060410000,RES,2326,LUAU ST,MESQUITE,751503826,38204500060410000,2326 LUAU ST MESQUITE TX 75150
1,38137500370290000,RES,1737,HIGHLAND ST,MESQUITE,751491534,38137500370290000,1737 HIGHLAND ST MESQUITE TX 75149
2,60259500060160000,RES,9999,AUTUMN WOOD DR,MESQUITE,75181,60259500060160000,9999 AUTUMN WOOD DR MESQUITE TX 75181
3,60259500050250000,RES,9925,AUTUMN WOOD DR,MESQUITE,75181,60259500050250000,9925 AUTUMN WOOD DR MESQUITE TX 75181
4,60259500060230000,RES,9999,RIPPLEWOOD DR,MESQUITE,75181,60259500060230000,9999 RIPPLEWOOD DR MESQUITE TX 75181


In [106]:
addresses = [df['FullAddress']]
addresses[0][0:300]

0             2326 LUAU ST MESQUITE TX 75150
1         1737 HIGHLAND ST MESQUITE TX 75149
2      9999 AUTUMN WOOD DR MESQUITE TX 75181
3      9925 AUTUMN WOOD DR MESQUITE TX 75181
4       9999 RIPPLEWOOD DR MESQUITE TX 75181
                       ...                  
295          1239 TOSCH LN MESQUITE TX 75149
296        1107 CYPRESS DR MESQUITE TX 75149
297       1427 CRAWFORD DR MESQUITE TX 75149
298         1106 POWELL RD MESQUITE TX 75149
299        1219 LANEMAR DR MESQUITE TX 75149
Name: FullAddress, Length: 300, dtype: object

In [105]:
try:
    client = Redfin()

    address = '2326 LUAU ST MESQUITE TX 75150'

    response = client.search(address)
    url = response['payload']['exactMatch']['url']
    initial_info = client.initial_info(url)

    property_id = initial_info['payload']['propertyId']
    mls_data = client.below_the_fold(property_id)

    # listing_id = initial_info['payload']['listingId']
    avm_details = client.avm_details(property_id, listing_id=None)

    pp.pprint(address)
    print("Response")
    pp.pprint(response)
    print("")

    print("URL")
    pp.pprint(url)
    print("")

    print("Initial Info")
    pp.pprint(initial_info)
    print("")

    print("Propery ID")
    pp.pprint(property_id)
    print("")

    print("MLS Data")
    pp.pprint(mls_data)
    print("")

    # pprint(f'listing id: {listing_id}')

    print("AVM Details")
    pp.pprint(avm_details)
    print("")

except Exception as e:
    pp.pprint(e)

'2326 LUAU ST MESQUITE TX 75150'
Response
{'errorMessage': 'Success',
 'payload': {'exactMatch': {'active': True,
                            'businessMarketIds': [11],
                            'claimedHome': False,
                            'countryCode': 'US',
                            'hasRental': False,
                            'id': '1_31901244',
                            'invalidMRS': False,
                            'name': '2326 Luau St',
                            'subName': 'Mesquite, TX, USA',
                            'type': '1',
                            'url': '/TX/Mesquite/2326-Luau-St-75150/home/31901244',
                            'urlV2': '/TX/Mesquite/2326-Luau-St-75150/home/31901244'},
             'extraResults': {},
             'hasFakeResults': False,
             'isGeocoded': False,
             'isRedfinServiced': False,
             'responseTime': 0,
             'sections': [{'name': 'Addresses',
                           'rows': [{'

In [114]:
client = Redfin()

successful = []
unsuccessful = []

for index, each in enumerate(addresses[0]):
    
    address = each
    url=''

    try:
        response = client.search(address)
        url = response['payload']['exactMatch']['url']
        successful.append((index, address))

    except Exception as e:
        unsuccessful.append((index, address))
        # print("*!*!*!*!*!*!*!*!*!")
        # print(e)
        # print(address)
        # print("*^*^*^^*^*^*")
    # initial_info = client.initial_info(url)

    # property_id = initial_info['payload']['propertyId']
    # mls_data = client.below_the_fold(property_id)

    # # listing_id = initial_info['payload']['listingId']
    # avm_details = client.avm_details(property_id)
    pp.pprint(url)

'/TX/Mesquite/2326-Luau-St-75150/home/31901244'
'/TX/Mesquite/1737-Highland-St-75149/home/31869232'
''
'/TX/Mesquite/9925-Autumn-Wood-Dr-75181/home/32116624'
''
'/TX/Mesquite/2900-Sidney-Dr-75150/home/31903524'
''
''
''
'/TX/Mesquite/1507-Richard-St-75149/home/31869683'
''
'/TX/Seagoville/806-Brandon-Dr-75159/home/30757826'
'/TX/Mesquite/302-S-Ebrite-St-75149/home/32125205'
'/TX/Mesquite/2411-Mesquite-Valley-Rd-75149/home/31851712'
'/TX/Seagoville/514-Payton-Ct-75159/home/30758435'
'/TX/Mesquite/2231-Schirra-Way-75150/home/31841559'
''
'/TX/Seagoville/1202-Bowers-Rd-75159/home/32044880'
''
'/TX/Mesquite/4733-Salem-Dr-75150/home/31814590'
'/TX/Mesquite/922-Autumn-Wood-Dr-75181/home/32116613'
'/TX/Mesquite/9924-Autumn-Wood-Dr-75181/home/32116611'
''
'/TX/Seagoville/503-W-Stark-Rd-75159/home/32131526'
''
'/TX/Mesquite/3124-Pecan-Xing-75181/home/32268270'
'/TX/Mesquite/837-W-Gross-St-75149/home/32125477'
'/TX/Seagoville/810-Brandon-Dr-75159/home/30757824'
'/TX/Mesquite/9929-Autumn-Wood-Dr-

KeyboardInterrupt: 

In [120]:
total = len(unsuccessful) + len(successful)
percentage = len(unsuccessful)/total * 100
print(percentage)
print(total)

6.86936936936937
888


In [123]:
for each in successful:
    print(each)

(0, '2326 LUAU ST MESQUITE TX 75150')
(1, '1737 HIGHLAND ST MESQUITE TX 75149')
(3, '9925 AUTUMN WOOD DR MESQUITE TX 75181')
(5, '2900 SIDNEY DR MESQUITE TX 75150')
(9, '1507 RICHARD ST MESQUITE TX 75149')
(11, '806 BRANDON DR SEAGOVILLE TX 75159')
(12, '302 S EBRITE ST MESQUITE TX 75149')
(13, '2411 MESQUITE VALLEY RD MESQUITE TX 75149')
(14, '514 PAYTON CT SEAGOVILLE TX 75159')
(15, '2231 SCHIRRA WAY  MESQUITE TX 75150')
(17, '1202 BOWERS RD SEAGOVILLE TX 75159')
(19, '4733 SALEM DR MESQUITE TX 75150')
(20, '922 AUTUMN WOOD DR MESQUITE TX 75181')
(21, '9924 AUTUMN WOOD DR MESQUITE TX 75181')
(23, '503 W STARK RD SEAGOVILLE TX 75159')
(25, '3124 PECAN CROSSING  MESQUITE TX 75181')
(26, '837 W GROSS ST MESQUITE TX 75149')
(27, '810 BRANDON DR SEAGOVILLE TX 75159')
(28, '9929 AUTUMN WOOD DR MESQUITE TX 75181')
(30, '9926 AUTUMN WOOD DR MESQUITE TX 75181')
(31, '1600 LIBERTY LN MESQUITE TX 75149')
(32, '1002 ARMSTRONG RD SEAGOVILLE TX 75159')
(34, '4341 SOARING STAR LN MESQUITE TX 75181'